In [12]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import os
import sys
from google.colab import drive
from tqdm import tqdm

In [9]:
#Conectar al drive local
drive.mount('/content/drive')

#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_helath'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

%run funciones.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# Etiquetas multiclase
class_labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

if __name__ == "__main__":

    path = 'data/data_despliegue'
    x, _, files = cargar_imagenes(path)
    x = np.array(x)

    modelo = tf.keras.models.load_model('data/salidas/best_model.keras')

    # Imprimir tamaño de entrada esperado
    print("Input shape esperado por el modelo:", modelo.input_shape)

    # Extraer tamaño esperado sin batch
    _, altura, ancho, canales = modelo.input_shape

    # Redimensionar imágenes al tamaño esperado
    x = np.array([cv2.resize(img, (ancho, altura), interpolation=cv2.INTER_AREA) for img in x])

    x = x.astype('float32') / 255.0
    pacientes = [name.rsplit('.', 1)[0] for name in files]

    prob = modelo.predict(x)
    y_pred = np.argmax(prob, axis=1)
    clases = [class_labels[i] for i in y_pred]

    resultados = pd.DataFrame({
        "paciente": pacientes,
        "clasificacion": clases
    })

    # Crear carpeta si no existe
    output_dir = 'data/salidas/resultados_despliegue'
    os.makedirs(output_dir, exist_ok=True)

    # Guardar archivo Excel
    resultados.to_excel(os.path.join(output_dir, 'clasificaciones_multiclase.xlsx'), index=False)
    print("Resultados exportados a 'clasificaciones_multiclase.xlsx'")

Cargando glioma: 100%|██████████| 100/100 [00:00<00:00, 161.58it/s]


Input shape esperado por el modelo: (None, 151, 150, 3)
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 287ms/step
Resultados exportados a 'clasificaciones_multiclase.xlsx'
